# Human Machine Interaction & Bias Mitigation

## Imports

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.inspection import permutation_importance, PartialDependenceDisplay
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_classification
from fairlearn.metrics import MetricFrame
from sklearn.metrics import accuracy_score, recall_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss

from tpot import TPOTClassifier
import shap
import lime
import lime.lime_tabular


from fairlearn.reductions import ExponentiatedGradient, EqualizedOdds


## Data Loading and Initial Analysis
- Loading the dataset using Pandas and performing an initial analysis to understand the basic properties. 
- Using descriptive statistics and visualizations to identify distributions, detect missing values, and spot outliers.
- Identifying the datatypes of the columns

In [8]:
df = pd.read_csv('patient_data.csv')

In [9]:
df.head(10)

,rs1047763,rs9282541,rs3827760,rs4988235,rs1801133,rs9374842,BMI,CardiovascularDisease
0,0,0,0,0,0,0,28.607859,0
1,0,0,1,1,0,0,26.651948,0
2,1,1,1,0,0,1,31.885502,0
3,0,0,1,0,0,0,29.353686,0
4,1,1,0,0,0,0,33.630251,0
5,0,0,0,0,0,0,28.243031,0
6,1,0,1,0,0,0,21.634838,0
7,1,1,1,1,0,0,36.809607,1
8,0,0,0,0,0,1,23.471339,0
9,0,0,0,0,1,1,23.231168,0


In [10]:
df.describe()

,rs1047763,rs9282541,rs3827760,rs4988235,rs1801133,rs9374842,BMI,CardiovascularDisease
count,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000
mean,0.433333,0.326667,0.490000,0.316667,0.286667,0.276667,28.899291,0.113333
std,0.496364,0.469778,0.500735,0.465953,0.452960,0.448098,5.171930,0.317529
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.798057,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25.292649,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,29.185791,0.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,32.131210,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,44.188743,1.000000


In [11]:
df.dtypes

rs1047763                  int64
rs9282541                  int64
rs3827760                  int64
rs4988235                  int64
rs1801133                  int64
rs9374842                  int64
BMI                      float64
CardiovascularDisease      int64
dtype: object

In [ ]:
# prints the number of distinct values in each column
for column in df.columns:
    num_distinct_values = df[column].nunique()
    print(f"Number of distinct values in {column}: {num_distinct_values}")

In [ ]:
df.isna().sum()

## Initial Data Cleaning
Getting an overview over the problems in the data, using the VS Code Extension Data Wrangler from Microsoft, then fix them.

- Preparing Data
- LabelEncoding categorical variables as needed, preparing data for Model 

NOT including any ML-method of imputation in this step as I haven't performed the train-test split yet and fitting the imputer on the whole df could risk data leakage -> will do this after EDA/Train-Test-Split.

In [ ]:
# Label encoding
label_encoder = LabelEncoder()
df_le = df

columns_to_encode = ['gender', 'Ethnicity', 'Socioeconomic Status', 'AppointmentNoshow']
for column in columns_to_encode:
    df_le[f'{column}_encoded'] = label_encoder.fit_transform(df_le[column])
df_le = df_le.drop(columns=columns_to_encode)

In [ ]:
# to be implemented after using DataWrangler VS Code extension


#should return df_clean

## Exploratory Data Analysis before Imputation 
Gettng an overview of the data -> how is the data spread out. 
For each column:
- Histogram: display the frequency of data points within specified bins, providing a visual representation of the distribution of a dataset.
- Density Plot:  visualize the distribution of data by estimating the probability density function, showing where values are concentrated -> represent probability distribution

### Histograms / Density Plots

In [ ]:
import seaborn as sns
sns.set_theme(style="whitegrid")

# Function to plot distribution of each column
def plot_distributions(df):
    for column in df.columns:
        plt.figure(figsize=(10, 5))
        
        # Histogram
        plt.subplot(1, 2, 1)
        sns.histplot(df[column], kde=False, bins=30)
        plt.title(f'Histogram of {column}')
        plt.xlabel(column)
        plt.ylabel('Frequency')
        
        # Density plot (KDE)
        plt.subplot(1, 2, 2)
        sns.kdeplot(df[column], fill=True)
        plt.title(f'Density Plot of {column}')
        plt.xlabel(column)
        plt.ylabel('Density')
        
        plt.tight_layout()
        plt.show()
        
        # Display basic statistics
        print(f'Statistics for {column}:')
        print(df[column].describe())
        print('\nSkewness:', df[column].skew())
        print('\nKurtosis:', df[column].kurtosis())
        print('\n')

# Plot distributions
plot_distributions(df_clean)

### Correlation Matrix

In [ ]:
corr_matrix = df_clean.corr()

# Plot the correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', cbar=True, square=True, linewidths=.5)
plt.title('Correlation Matrix Heatmap')
plt.show()

## Train-Test Split
Using the train_test_split from sklearn.model_selection to split the data
- split into train and interim test set
- split interim test set into val and test set
- export test set to csv
- delete interim_test_set and test_set from notebook -> helps ensure prevention of data leakage

In [ ]:
# First split: 70-30 train-test split, with interim test set
train_set, interim_test_set = train_test_split(df, test_size=0.3, random_state=42)

In [ ]:
# splitting the train set into X_train and y_train
X_train = train_set.drop('targetvariable', axis=1)
y_train = train_set['targetvariable']

In [ ]:
# Second split: 50-50 validation-test split
val_set, test_set = train_test_split(interim_test_set, test_size=0.5, random_state=42)

In [ ]:
# splitting the validation set into X_val and y_val
X_val = train_set.drop('targetvariable', axis=1)
y_val = train_set['targetvariable']

In [ ]:
# Export the test set to a CSV file & delete test set and interim test set
test_set.to_csv('test_set.csv', index=False)

del test_set, interim_test_set

## Imputation

Trying different methods of imputation (KNN & RandomForest Imputation with Iterative Imputer) -> see which provides the better result (= the lower mean squared error)

In [ ]:
def evaluate_imputation(X, y, imputer, model):
    pipeline = Pipeline([
        ('imputer', imputer),
        ('scaler', StandardScaler()),  # Scaling is applied after imputation
        ('regressor', model)
    ])
    
    # Set up cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    # Cross-validate the pipeline
    scores = cross_val_score(pipeline, X, y, scoring='neg_mean_squared_error', cv=kf)
    
    # Return average MSE
    return np.mean(-scores)

In [ ]:
# Initialize the KNN imputer
knn_imputer = KNNImputer(n_neighbors=5)

# Initialize the Iterative Imputer using RandomForest
rf_imputer = IterativeImputer(estimator=RandomForestRegressor(n_estimators=10, random_state=42), random_state=42)

# Initialize the regression model
model = RandomForestRegressor(n_estimators=100, random_state=42)

In [ ]:
# Evaluate KNN imputer
mse_knn = evaluate_imputation(X_train, y_train, knn_imputer, model)
print(f"KNN Imputer MSE: {mse_knn}")

# Evaluate Random Forest imputer (IterativeImputer)
mse_rf = evaluate_imputation(X_train, y_train, rf_imputer, model)
print(f"Random Forest Imputer MSE: {mse_rf}")

# Determine the best imputer
best_imputer = 'KNN' if mse_knn < mse_rf else 'Random Forest'
print(f"Best imputer selected based on cross-validated MSE: {best_imputer}")

### Fitting the selected Imputer on the Training & Validation Set

In [ ]:
### TODO - Implement the best imputer - delete other imputer

In [ ]:
# Fit KNN imputer on the training data
knn_imputer = KNNImputer(n_neighbors=5)
X_train_imputed = knn_imputer.fit_transform(X_train)
X_val_imputed = knn_imputer.transform(X_val)

In [ ]:
# Fit Iterative Imputer using RandomForest on the training data
rf_imputer = IterativeImputer(estimator=RandomForestRegressor(n_estimators=10, random_state=42), random_state=42)
X_train_imputed = rf_imputer.fit_transform(X_train)
X_val_imputed = rf_imputer.transform(X_val)

## Model Selection & Training with TPOT
Using TPOTClassifier (Tree-based Pipeline Optimization Tool) to automate the selection and training of the best predictive model based on the cleaned training dataset. This tool explores various models and hyperparameter settings to find the optimal solution. As TPOT can also Impute missing values, I am trying out these two things:
- TPOT Classifier on Data without Imputation
- TPOT Classifier on Data with Imputation from previous step
In order to find out which has better scores.



### TPOT with dataframe with NaN

In [ ]:
# Instantiate and train the TPOT classifier
tpot = TPOTClassifier(generations=5, population_size=50, verbosity=2, scoring='accuracy', random_state=42)
tpot.fit(X_train, y_train)

# Evaluate the classifier on the validation set
print("Validation Accuracy: ", tpot.score(X_val, y_val))

# Export the best model
tpot.export('tpot_best_model.py')

### TPOT with imputed dataframes

In [ ]:
# Instantiate and train the TPOT classifier
tpot = TPOTClassifier(generations=5, population_size=50, verbosity=2, scoring='accuracy', random_state=42)
tpot.fit(X_train_imputed, y_train)

# Evaluate the classifier on the validation set
print("Validation Accuracy: ", tpot.score(X_val_imputed, y_val))

# Export the best model
tpot.export('tpot_best_model.py')

### Saving best pipeline from TPOT

In [ ]:
best_pipeline = tpot.fitted_pipeline_

## Model Evaluation
In this step, I analyse the output the best model from the TPOT Classifier using various metrics and visualize results.

### Accuracy

In [ ]:
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy: {accuracy:.2f}")

### Classification Report

In [ ]:
class_report = classification_report(y_val, y_pred)
print("Classification Report:\n", class_report)

### Confusion Matrix

In [ ]:
conf_matrix = confusion_matrix(y_val, y_pred)

sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted Neg', 'Predicted Pos'], yticklabels=['Actual Neg', 'Actual Pos'])
plt.xlabel('Predicted Labels')
plt.ylabel('Actual Labels')
plt.title('Confusion Matrix')
plt.show()

#### ROC-AUC Score

In [ ]:
y_prob = model.predict_proba(X_val)[:, 1]  # get the probability of the positive class
roc_auc = roc_auc_score(y_val, y_prob)
print(f"ROC-AUC Score: {roc_auc:.2f}")

### Classifying False-Positives & False-Negatives

In [ ]:

data = {'Actual': y_val, 'Predicted': y_pred}
df = pd.DataFrame(data)

# Identify false positives and false negatives
false_positives_val = df[(df['Actual'] == 0) & (df['Predicted'] == 1)]
false_negatives_val = df[(df['Actual'] == 1) & (df['Predicted'] == 0)]


## Model Explanation

### Global Methods

#### SHAP Summary Plot

In [ ]:
explainer = shap.Explainer(exported_pipeline.predict, X_train)
shap_values = explainer(X_test)

# the waterfall_plot shows how we get from shap_values.base_values to model.predict(X)[sample_ind]
shap.plots.beeswarm(shap_values)

#### Permutation Feature Importance

In [ ]:
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import numpy as np

# Assuming 'model' is already trained and 'X_val', 'y_val' are defined
result = permutation_importance(model, X_val, y_val, n_repeats=10, random_state=42, n_jobs=-1)

# Get sorted importances
sorted_idx = result.importances_mean.argsort()

# Select top N features
top_n = 15  # Adjust N based on your preference
sorted_idx_top = sorted_idx[-top_n:]

fig, ax = plt.subplots(figsize=(8, 8))
ax.boxplot(result.importances[sorted_idx_top].T, vert=False, labels=X_val.columns[sorted_idx_top])
ax.set_title("Permutation Importance of Top Features")
ax.set_xlabel("Importance")
plt.show()

### Local Methods

#### LIME

In [ ]:


# Create a LimeTabularExplainer
# Ensure your data is in a suitable format, like a numpy array
explainer = lime.lime_tabular.LimeTabularExplainer(
    X_train.values, 
    feature_names=X_train.columns.tolist(),
    class_names=['No CHD', 'CHD'],  # Update these as per your class labels
    mode='classification'
)

# Choose an instance from your validation set to explain
i = 10  # Index of the instance in the validation set
exp = explainer.explain_instance(X_val.iloc[i], model.predict_proba, num_features=5)

# Display the explanation in a Jupyter Notebook
exp.show_in_notebook(show_table=True)

#### SHAP Values

## Bias Identification and Mitigation
I am using Fairlearn, a library that was initially developed by researchers at Microsoft. It is now mainained as an open-source project to aid in assessing and mitigating fairness issues in AI. 

In [ ]:
def evaluate_fairness_metrics(y_true, y_pred, sensitive_features, sensitive_feature_name):
    # Extract the specific sensitive feature
    sensitive_feature_data = sensitive_features[sensitive_feature_name]

    # Define the metrics you want to check
    metrics = {
        'accuracy': accuracy_score,
        'recall': recall_score
    }

    # Compute metrics
    metric_frame = MetricFrame(metrics=metrics,
                               y_true=y_true,
                               y_pred=y_pred,
                               sensitive_features=sensitive_feature_data)

    # Print the results
    print("Overall metrics:")
    print(metric_frame.overall)
    print("\nMetrics by group:")
    print(metric_frame.by_group)

evaluate_fairness_metrics(y_val, y_pred, X_val, 'Ethnicity')

### Demographic Parity / Disparity
Measures if the decision boundary of the classifier does not vary between groups = groups should receive positive outcomes at equal rates.

In [ ]:
def calculate_demographic_parity(predictions, sensitive_features):
    """ Calculate demographic parity disparity"""
    # Convert inputs to pandas Series if they aren't already
    if not isinstance(predictions, pd.Series):
        predictions = pd.Series(predictions)
    if not isinstance(sensitive_features, pd.Series):
        sensitive_features = pd.Series(sensitive_features)
    
    # Combine into a single DataFrame
    data = pd.DataFrame({
        'predictions': predictions,
        'sensitive_features': sensitive_features
    })
    
    # Group by the sensitive feature and calculate the mean outcome
    parity = data.groupby('sensitive_features')['predictions'].mean()
    disparity = np.abs(parity - parity.mean()).max()
    return disparity

dp_disparity = calculate_demographic_parity(y_pred, X_val['Ethnicity'])
print("Demographic Parity Disparity:", dp_disparity)

- Value = 0: This is the ideal value. It indicates that all groups have the same probability of receiving a positive outcome, fulfilling the criterion of demographic parity perfectly. There is no disparity between the groups in terms of the positive outcome rate.
- Value > 0: A nonzero value indicates a disparity in the positive outcome rates across different groups. The closer this value is to 0, the lesser the disparity:
	- Low Disparity (e.g., values closer to 0, like 0.05 or 0.1): Suggests a relatively fair model where the differences in positive outcome rates between groups are minimal.
	- Moderate Disparity (e.g., around 0.2 to 0.3): Indicates a noticeable difference in how groups are treated by the model, which could be a cause for concern and may necessitate further investigation or adjustment of the model.
	- High Disparity (e.g., values approaching 0.5 or higher): Represents significant unfairness, with some groups being substantially more likely to receive positive outcomes than others. This level of disparity is typically unacceptable in practice and requires immediate attention to correct the bias.

#### Equal Opportunity Disparity


In [ ]:
def calculate_equal_opportunity(predictions, sensitive_features, ground_truth):
    """
    Calculate the Equal Opportunity metric.
    """
    # Combine predictions and ground truth
    data = pd.DataFrame({'predictions': predictions, 'ground_truth': ground_truth, 'sensitive_features': sensitive_features})
    # Function to calculate True Positive Rate
    def true_positive_rate(group):
        tp = ((group['predictions'] == 1) & (group['ground_truth'] == 1)).sum()
        actual_positives = (group['ground_truth'] == 1).sum()
        return tp / actual_positives if actual_positives > 0 else 0
    
    # Calculate TPR by group
    tpr = data.groupby('sensitive_features').apply(true_positive_rate)
    disparity = np.abs(tpr - tpr.mean()).max()
    return disparity

eo_disparity = calculate_equal_opportunity(pd.Series(y_pred, name='predictions'), pd.Series(X_val['Ethnicity'], name='sensitive_features'), pd.Series(y_val, name='ground_truth'))

print("Equal Opportunity Disparity:", eo_disparity)

- Value = 0: This value indicates perfect fairness in terms of equal opportunity. It means that all groups have the same true positive rate, i.e., the model is equally likely to correctly predict positive outcomes across all sensitive groups.
- Value > 0: Indicates that there’s a difference in the true positive rates among the groups:
  - Low Disparity (e.g., values closer to 0, like 0.05 or 0.1): Suggests that the model is relatively fair, with minimal differences in how accurately it predicts positive outcomes across different groups.
	- Moderate Disparity (e.g., around 0.15 to 0.25): This level of disparity points to a moderate level of unfairness. The model is less accurate in predicting positive outcomes for some groups compared to others, which might raise concerns depending on the application context.
	- High Disparity (e.g., values approaching 0.3 or higher): Indicates a significant level of unfairness, showing that the model performs considerably better for certain groups than for others when predicting positive outcomes. This is generally unacceptable and requires adjustments to the model or its training data.

#### Equality of Odds

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix

def calculate_equality_of_odds(predictions, sensitive_features, ground_truth):
    """
    Calculate the Equality of Odds metric.
    It includes both True Positive Rate (TPR) and False Positive Rate (FPR) disparities.
    """
    data = pd.DataFrame({
        'predictions': predictions,
        'ground_truth': ground_truth,
        'sensitive_features': sensitive_features
    })
    
    def odds_rates(group):
        tn, fp, fn, tp = confusion_matrix(group['ground_truth'], group['predictions']).ravel()
        tpr = tp / (tp + fn) if (tp + fn) != 0 else 0
        fpr = fp / (fp + tn) if (fp + tn) != 0 else 0
        return pd.Series({'TPR': tpr, 'FPR': fpr})
    
    odds = data.groupby('sensitive_features').apply(odds_rates)
    tpr_disparity = np.abs(odds['TPR'] - odds['TPR'].mean()).max()
    fpr_disparity = np.abs(odds['FPR'] - odds['FPR'].mean()).max()
    return tpr_disparity, fpr_disparity


tpr_disparity, fpr_disparity = calculate_equality_of_odds(y_pred, X_val['Ethnicity'], y_val)
print("TPR Disparity:", tpr_disparity)
print("FPR Disparity:", fpr_disparity)

#### Overall Accuracy Disparity

In [ ]:
def calculate_overall_accuracy_equality(predictions, sensitive_features, ground_truth):
    """
    Calculate Overall Accuracy Equality across different groups.
    """
    data = pd.DataFrame({
        'predictions': predictions,
        'ground_truth': ground_truth,
        'sensitive_features': sensitive_features
    })
    
    def accuracy(group):
        correct = (group['predictions'] == group['ground_truth']).sum()
        total = group.shape[0]
        return correct / total
    
    accuracies = data.groupby('sensitive_features').apply(accuracy)
    accuracy_disparity = np.abs(accuracies - accuracies.mean()).max()
    return accuracy_disparity


accuracy_disparity = calculate_overall_accuracy_equality(y_pred, X_val['Ethnicity'], y_val)
print("Overall Accuracy Disparity:", accuracy_disparity)

#### SMOTE (Oversampling)
Oversamples the minority class with synthetic samples.

In [7]:
# Applying SMOTE
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X_train, y_train)

print(f'Original dataset shape {Counter(y_train)}')
print(f'Resampled dataset shape {Counter(y_smote)}')

NameError: name 'X_train' is not defined

#### NearMiss (Undersampling)
Undersamples instances that are particularly close to instances of the minority class.

In [ ]:
# Applying NearMiss
nm = NearMiss(version=1)
X_nm, y_nm = nm.fit_resample(X_train, y_train)

print(f'Original dataset shape {Counter(y_train)}')
print(f'Resampled dataset shape {Counter(y_nm)}')

#### Class-weight Balancing
Balances the weight of the individual classes